<a href="https://colab.research.google.com/github/Null2648/google-colab/blob/main/10_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boosting 이란?
 - 여러개의 약한 Decision Tree를 조합해서 사용하는 Ensemble 기법중 하나이다.
 - 즉, 약한 예측 모델들의 학습 에러에 가중치를 두고, 순차적으로 다음 학습 모델에 반영하여 강한 예측모델을 만드는 것이다.


# GBM(Grandient Boosting)
 - GBM은 여러 개의 weak learner를 순차적으로 학습, 예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식
 - 가중치의 업데이트 방법은 경사하강법(Grandient Descent)을 사용한다.


# XGBoost 란?
 - XGBoost는 Extreme Gradient Boosting의 약자이다.
 - Boosting 기법을 이용하여 구현한 알고리즘은 Graiednt Boost가 대표적인데 이 알고리즘을 병렬 합습이 지원되도록 구현한 라이브러리가 XGBoost이다.
 - Regression, Classifier 문제를 모두 지원하며, 성능과 자원 효율이 좋아서 인기 있는 알고리즘이다.
 - XGBoost의 장점
  1. GBM 대비 빠른 수행시간: 병렬 처리로 학습, 분류속도가 빠른다.
  2. 과적합 규제(Regularization): 표준 GBM 경우 과적합 규제기능이 없으나, XGBoost는 자체에 과적합 규제 기능으로 강한 내구성을 지닌다.
  3. 분류과 회귀 영역에서 뛰어난 예측 성능을 발휘한다.
    - 즉, CART(Classifier and Regression Tree)앙상블 모델을 사용한다.
  4. Early Stopping(조기 종료) 기능이 있다.
  5. 다양한 옵션을 제공하며 Customizing이 용이하다. 

In [2]:
import numpy as np
import io
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family = 'malgun gothic')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

#차트 스타일 지정
sns.set(font='malgun gothic', rc={'axes.unicode_minus':False}, style = 'darkgrid')
plt.rc('figure', figsize=(10, 8))


warnings.filterwarnings('ignore')

# 로지스틱 회귀와 평가지표

## 데이터셋 - 위스콘신 유방암 예측

In [6]:
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()
# X
# y